##### Usecase:Get sales data by products
###### Solutions are given for following problem statements
###### Problem statement-1:Get sales data for products per day
###### Problem statement-2:Get sales data for products per month
###### Problem statement-3:Get sales data for products per year
###### Problem statement-4:Get monthly sales data from most sold products
###### Problem statement-5:Get third and final quarters of sales data from Top5 revenue generating products for each month
###### Problem statement-6:Get sales data for each product category
###### Problem statement-7:Get monthly sales data from Top5 revenue generating categories
###### Problem statement-8:Get highest product price for each category
###### Problem statement-9:Get the sales data of best-selling and the second best-selling products in every category
###### Problem statement-10:Get TOP5 highest selling product for each year

In [2]:
from pyspark.sql import SQLContext, Row
from pyspark.sql import HiveContext
from pyspark.sql.functions import round,sum,avg,count,substring
hiveContext = HiveContext(sc)
sqlContext.sql("set spark.sql.shuffle.partitions=10")

Out[1]: DataFrame[key: string, value: string]

In [3]:
hiveContext.sql("create database retail_db")
hiveContext.sql("use retail_db")

In [4]:
hiveContext.sql("create table orders(order_id int,order_date string,order_customer_id int, order_status string) row format delimited fields terminated by ','")
hiveContext.sql("create table order_items(order_item_id int, order_item_order_id int, order_item_product_id  int, order_item_quantity int,order_item_subtotal float,order_item_product_price float) row format delimited fields terminated by ','")
hiveContext.sql("create table customers(customer_id int, customer_fname string, customer_lname string, customer_email string,customer_password string,customer_street string,customer_city string, customer_state string, customer_zipcode string) row format delimited fields terminated by ','")
hiveContext.sql("create table products(product_id int, product_category_id int, product_name string, product_desc string, product_price float, product_image string) \ 
                                                           row format delimited fields terminated by ';'")

In [5]:
hiveContext.sql("load data local inpath 'dbfs:/FileStore/tables/retaildbtext/Orders.txt' into table orders")
hiveContext.sql("load data local inpath 'dbfs:/FileStore/tables/retaildbtext/Order_Items.txt' into table order_items")
hiveContext.sql("load data local inpath 'dbfs:/FileStore/tables/retaildbtext/Customers.txt' into table customers")
hiveContext.sql("load data local inpath 'dbfs:/FileStore/tables/retaildbtext/Products.txt' into table customers")

In [6]:
hiveContext.sql("show tables").show()

+---------+-----------+-----------+
 database| tableName|isTemporary|
+---------+-----------+-----------+
retail_db| categories| false|
retail_db| customers| false|
retail_db|departments| false|
retail_db|order_items| false|
retail_db| orders| false|
retail_db| products| false|
+---------+-----------+-----------+

In [7]:
hiveContext.sql("select * from orders").show(5,False)

+--------+-----------------------+-----------------+---------------+
order_id|order_date |order_customer_id|order_status |
+--------+-----------------------+-----------------+---------------+
1 |2013-07-25 00:00:00.000|11599 |CLOSED |
2 |2013-07-25 00:00:00.000|256 |PENDING_PAYMENT|
3 |2013-07-25 00:00:00.000|12111 |COMPLETE |
4 |2013-07-25 00:00:00.000|8827 |CLOSED |
5 |2013-07-25 00:00:00.000|11318 |COMPLETE |
+--------+-----------------------+-----------------+---------------+
only showing top 5 rows

In [8]:
hiveContext.sql("select * from orders").printSchema()

root
-- order_id: string (nullable = true)
-- order_date: string (nullable = true)
-- order_customer_id: string (nullable = true)
-- order_status: string (nullable = true)

In [9]:
hiveContext.sql("select * from order_items").show(5,False)


+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
1 |1 |957 |1 |299.98 |299.98 |
2 |2 |1073 |1 |199.99 |199.99 |
3 |2 |502 |5 |250.0 |50.0 |
4 |2 |403 |1 |129.99 |129.99 |
5 |4 |897 |2 |49.98 |24.99 |
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
only showing top 5 rows

In [10]:
hiveContext.sql("select * from order_items").printSchema()

root
-- order_item_id: string (nullable = true)
-- order_item_order_id: string (nullable = true)
-- order_item_product_id: string (nullable = true)
-- order_item_quantity: string (nullable = true)
-- order_item_subtotal: string (nullable = true)
-- order_item_product_price: string (nullable = true)

In [11]:
hiveContext.sql("select * from customers").show(5,False)

+-----------+--------------+--------------+--------------+-----------------+-----------------------+-------------+--------------+----------------+
customer_id|customer_fname|customer_lname|customer_email|customer_password|customer_street |customer_city|customer_state|customer_zipcode|
+-----------+--------------+--------------+--------------+-----------------+-----------------------+-------------+--------------+----------------+
1 |Richard |Hernandez |XXXXXXXXX |XXXXXXXXX |6303 Heather Plaza |Brownsville |TX |78521 |
2 |Mary |Barrett |XXXXXXXXX |XXXXXXXXX |9526 Noble Embers Ridge|Littleton |CO |80126 |
3 |Ann |Smith |XXXXXXXXX |XXXXXXXXX |3422 Blue Pioneer Bend |Caguas |PR |00725 |
4 |Mary |Jones |XXXXXXXXX |XXXXXXXXX |8324 Little Common |San Marcos |CA |92069 |
5 |Robert |Hudson |XXXXXXXXX |XXXXXXXXX |10 Crystal River Mall |Caguas |PR |00725 |
+-----------+--------------+--------------+--------------+-----------------+-----------------------+-------------+--------------+----------------+
only showing top 5 rows

In [12]:
hiveContext.sql("select * from customers").printSchema()

root
-- customer_id: string (nullable = true)
-- customer_fname: string (nullable = true)
-- customer_lname: string (nullable = true)
-- customer_email: string (nullable = true)
-- customer_password: string (nullable = true)
-- customer_street: string (nullable = true)
-- customer_city: string (nullable = true)
-- customer_state: string (nullable = true)
-- customer_zipcode: string (nullable = true)

In [13]:
hiveContext.sql("select * from products").show(5)

+----------+-------------------+--------------------+-------------------+-------------+--------------------+
product_id|product_category_id| product_name|product_description|product_price| product_image|
+----------+-------------------+--------------------+-------------------+-------------+--------------------+
 1| 2|Quest Q64 10 FT. ...| null| 59.98|http://images.acm...|
 2| 2|Under Armour Men'...| null| 129.99|http://images.acm...|
 3| 2|Under Armour Men'...| null| 89.99|http://images.acm...|
 4| 2|Under Armour Men'...| null| 89.99|http://images.acm...|
 5| 2|Riddell Youth Rev...| null| 199.99|http://images.acm...|
+----------+-------------------+--------------------+-------------------+-------------+--------------------+
only showing top 5 rows

In [14]:
hiveContext.sql("select * from products").printSchema()

root
-- product_id: string (nullable = true)
-- product_category_id: string (nullable = true)
-- product_name: string (nullable = true)
-- product_description: string (nullable = true)
-- product_price: string (nullable = true)
-- product_image: string (nullable = true)

###### Problem statement-1:Get sales data for products per day

In [16]:
productsSalesDataPerDay = hiveContext.sql('''select substring(otl.order_date,0,10) as orderdate,pdtl.product_name,
                                   round(sum(oitl.order_item_subtotal),2) as Totalrevenue_perday,  
                                   count(oitl.order_item_subtotal) as NumberOforders_perday, 
                                   round(avg(oitl.order_item_subtotal),2) as avgsales_perday 
                                   from orders otl join order_items oitl on otl.order_id=oitl.order_item_order_id
                                   join products pdtl on oitl.order_item_product_id = pdtl.product_id
                                   group by orderdate,pdtl.product_name 
                                   order by orderdate,pdtl.product_name''')
productsSalesDataPerDay.coalesce(1).write.csv("/FileStore/tables/productsSalesDataPerDay", compression="none", header ='true')
productsSalesDataPerDay.show(20,False)

+----------+---------------------------------------------+-------------------+---------------------+---------------+
orderdate |product_name |Totalrevenue_perday|NumberOforders_perday|avgsales_perday|
+----------+---------------------------------------------+-------------------+---------------------+---------------+
2013-07-25|Bag Boy Beverage Holder |99.96 |1 |99.96 |
2013-07-25|Bowflex SelectTech 1090 Dumbbells |599.99 |1 |599.99 |
2013-07-25|Bridgestone e6 Straight Distance NFL Carolina|63.98 |1 |63.98 |
2013-07-25|Bridgestone e6 Straight Distance NFL San Dieg|95.97 |1 |95.97 |
2013-07-25|Bridgestone e6 Straight Distance NFL Tennesse|127.96 |1 |127.96 |
2013-07-25|Cleveland Golf Women's 588 RTX CB Satin Chrom|239.98 |2 |119.99 |
2013-07-25|Clicgear 8.0 Shoe Brush |19.98 |1 |19.98 |
2013-07-25|Diamondback Women's Serene Classic Comfort Bi|9599.36 |32 |299.98 |
2013-07-25|Elevation Training Mask 2.0 |319.96 |1 |319.96 |
2013-07-25|Field & Stream Sportsman 16 Gun Fire Safe |10799.46 |27 |399.98 |
2013-07-25|Glove It Imperial Golf Towel |79.95 |1 |79.95 |
2013-07-25|Glove It Urban Brick Golf Towel |79.95 |1 |79.95 |
2013-07-25|Glove It Women's Mod Oval 3-Zip Carry All Gol|21.99 |1 |21.99 |
2013-07-25|Glove It Women's Mod Oval Golf Glove |79.96 |1 |79.96 |
2013-07-25|GolfBuddy VT3 GPS Watch |199.99 |1 |199.99 |
2013-07-25|LIJA Women's Button Golf Dress |216.0 |2 |108.0 |
2013-07-25|MDGolf Pittsburgh Penguins Putter |239.97 |1 |239.97 |
2013-07-25|Merrell Men's All Out Flash Trail Running Sho|219.98 |2 |109.99 |
2013-07-25|Nike Men's CJ Elite 2 TD Football Cleat |5589.57 |43 |129.99 |
2013-07-25|Nike Men's Dri-FIT Victory Golf Polo |5100.0 |39 |130.77 |
+----------+---------------------------------------------+-------------------+---------------------+---------------+
only showing top 20 rows

###### Problem statement-2: Get sales data for products per month

In [18]:
productsSalesDataPerMonth = hiveContext.sql('''select substring(otl.order_date,0,7) as ordermonth,pdtl.product_name,
                                   round(sum(oitl.order_item_subtotal),2) as Totalrevenue_permonth,  
                                   count(oitl.order_item_subtotal) as NumberOforders_permonth, 
                                   round(avg(oitl.order_item_subtotal),2) as avgsales_permonth 
                                   from orders otl join order_items oitl on otl.order_id=oitl.order_item_order_id
                                   join products pdtl on oitl.order_item_product_id = pdtl.product_id
                                   group by ordermonth,pdtl.product_name 
                                   order by ordermonth,pdtl.product_name''')
productsSalesDataPerMonth.coalesce(1).write.csv("/FileStore/tables/productsSalesDataPerMonth", compression="none", header ='true')
productsSalesDataPerMonth.show(20,False)

+----------+---------------------------------------------+---------------------+-----------------------+-----------------+
ordermonth|product_name |Totalrevenue_permonth|NumberOforders_permonth|avgsales_permonth|
+----------+---------------------------------------------+---------------------+-----------------------+-----------------+
2013-07 |Bag Boy Beverage Holder |549.78 |7 |78.54 |
2013-07 |Bowflex SelectTech 1090 Dumbbells |599.99 |1 |599.99 |
2013-07 |Bridgestone e6 Straight Distance NFL Carolina|1055.67 |13 |81.21 |
2013-07 |Bridgestone e6 Straight Distance NFL San Dieg|575.82 |8 |71.98 |
2013-07 |Bridgestone e6 Straight Distance NFL Tennesse|319.9 |4 |79.98 |
2013-07 |Brooks Women's Ghost 6 Running Shoe |179.98 |1 |179.98 |
2013-07 |Cleveland Golf Collegiate My Custom Wedge 588|209.99 |1 |209.99 |
2013-07 |Cleveland Golf Women's 588 RTX CB Satin Chrom|359.97 |3 |119.99 |
2013-07 |Clicgear 8.0 Shoe Brush |129.87 |5 |25.97 |
2013-07 |Clicgear Rovic Cooler Bag |639.84 |4 |159.96 |
2013-07 |Columbia Men's PFG Anchor Tough T-Shirt |510.0 |5 |102.0 |
2013-07 |Diamondback Boys' Insight 24 Performance Hybr|299.99 |1 |299.99 |
2013-07 |Diamondback Women's Serene Classic Comfort Bi|95993.6 |320 |299.98 |
2013-07 |ENO Atlas Hammock Straps |719.76 |9 |79.97 |
2013-07 |Elevation Training Mask 2.0 |319.96 |1 |319.96 |
2013-07 |Field & Stream Sportsman 16 Gun Fire Safe |155592.22 |389 |399.98 |
2013-07 |Fitbit The One Wireless Activity & Sleep Trac|399.8 |1 |399.8 |
2013-07 |Garmin Approach S3 Golf GPS Watch |249.99 |1 |249.99 |
2013-07 |Glove It Imperial Golf Towel |143.91 |2 |71.96 |
2013-07 |Glove It Urban Brick Golf Towel |415.74 |7 |59.39 |
+----------+---------------------------------------------+---------------------+-----------------------+-----------------+
only showing top 20 rows

###### Problem statement-3: Get sales data  for products per year

In [20]:
productsSalesDataPerYear = hiveContext.sql('''select substring(otl.order_date,0,4) as orderyear,pdtl.product_name,
                                   round(sum(oitl.order_item_subtotal),2) as Totalrevenue_peryear,  
                                   count(oitl.order_item_subtotal) as NumberOforders_peryear, 
                                   round(avg(oitl.order_item_subtotal),2) as avgsales_peryear 
                                   from orders otl join order_items oitl on otl.order_id=oitl.order_item_order_id
                                   join products pdtl on oitl.order_item_product_id = pdtl.product_id
                                   group by orderyear, pdtl.product_name 
                                   order by orderyear, pdtl.product_name''')
productsSalesDataPerYear.coalesce(1).write.csv("/FileStore/tables/productsSalesDataPerYear", compression="none", header ='true')
productsSalesDataPerYear.show(20,False)

+---------+---------------------------------------------+--------------------+----------------------+----------------+
orderyear|product_name |Totalrevenue_peryear|NumberOforders_peryear|avgsales_peryear|
+---------+---------------------------------------------+--------------------+----------------------+----------------+
2013 |Bag Boy Beverage Holder |10195.92 |126 |80.92 |
2013 |Bag Boy M330 Push Cart |6799.15 |32 |212.47 |
2013 |Bowflex SelectTech 1090 Dumbbells |4199.93 |7 |599.99 |
2013 |Bridgestone e6 Straight Distance NFL Carolina|14491.47 |147 |98.58 |
2013 |Bridgestone e6 Straight Distance NFL San Dieg|12124.21 |136 |89.15 |
2013 |Bridgestone e6 Straight Distance NFL Tennesse|11132.52 |109 |102.13 |
2013 |Brooks Women's Ghost 6 Running Shoe |8729.03 |26 |335.73 |
2013 |Bushnell Pro X7 Jolt Slope Rangefinder |2399.96 |4 |599.99 |
2013 |Cleveland Golf Collegiate My Custom Wedge 588|6089.71 |29 |209.99 |
2013 |Cleveland Golf Women's 588 RTX CB Satin Chrom|3359.72 |28 |119.99 |
2013 |Clicgear 8.0 Shoe Brush |4185.81 |132 |31.71 |
2013 |Clicgear Rovic Cooler Bag |14956.26 |115 |130.05 |
2013 |Columbia Men's PFG Anchor Tough T-Shirt |11940.0 |125 |95.52 |
2013 |Diamondback Boys' Insight 24 Performance Hybr|4199.86 |14 |299.99 |
2013 |Diamondback Girls' Clarity 24 Hybrid Bike 201|3599.88 |12 |299.99 |
2013 |Diamondback Women's Serene Classic Comfort Bi|1832277.84 |6108 |299.98 |
2013 |ENO Atlas Hammock Straps |13285.57 |150 |88.57 |
2013 |Elevation Training Mask 2.0 |9678.79 |38 |254.7 |
2013 |Field & Stream Sportsman 16 Gun Fire Safe |3076246.18 |7691 |399.98 |
2013 |Fitbit The One Wireless Activity & Sleep Trac|6496.75 |22 |295.31 |
+---------+---------------------------------------------+--------------------+----------------------+----------------+
only showing top 20 rows

###### Problem statement-4:Get monthly sales data from most sold products between 2013-July and 2014-July

In [22]:
productsSalesDataformostsold = hiveContext.sql('''select * from (select substring(otl.order_date,0,7) as ordermonth,pdtl.product_name,
                                   sum(oitl.order_item_quantity) order_quantity,
                                   rank() over(partition by substring(otl.order_date,0,7) order by sum(oitl.order_item_quantity) desc) ranking_order
                                   from orders otl,order_items oitl,products pdtl where otl.order_id=oitl.order_item_order_id
                                   and oitl.order_item_product_id = pdtl.product_id
                                   group by substring(otl.order_date,0,7),pdtl.product_name) s 
                                   WHERE ranking_order=1 ORDER BY 1''')
productsSalesDataformostsold.coalesce(1).write.csv("/FileStore/tables/productsSalesDataformostsold", compression="none", header ='true')
productsSalesDataformostsold.show(20,False)

+----------+--------------------------------+--------------+-------------+
ordermonth|product_name |order_quantity|ranking_order|
+----------+--------------------------------+--------------+-------------+
2013-07 |Perfect Fitness Perfect Rip Deck|1710.0 |1 |
2013-08 |Perfect Fitness Perfect Rip Deck|6089.0 |1 |
2013-09 |Perfect Fitness Perfect Rip Deck|6153.0 |1 |
2013-10 |Perfect Fitness Perfect Rip Deck|5699.0 |1 |
2013-11 |Perfect Fitness Perfect Rip Deck|6745.0 |1 |
2013-12 |Perfect Fitness Perfect Rip Deck|5983.0 |1 |
2014-01 |Perfect Fitness Perfect Rip Deck|6340.0 |1 |
2014-02 |Perfect Fitness Perfect Rip Deck|6315.0 |1 |
2014-03 |Perfect Fitness Perfect Rip Deck|6115.0 |1 |
2014-04 |Perfect Fitness Perfect Rip Deck|5959.0 |1 |
2014-05 |Perfect Fitness Perfect Rip Deck|6014.0 |1 |
2014-06 |Perfect Fitness Perfect Rip Deck|5873.0 |1 |
2014-07 |Perfect Fitness Perfect Rip Deck|4703.0 |1 |
+----------+--------------------------------+--------------+-------------+

###### Problem statement-5:Get third and final quarters of sales data from Top5 revenue generating products for each month

In [24]:
productsSalesDatafortop5revenue = hiveContext.sql('''select * from (select substring(otl.order_date,0,7) as ordermonth,pdtl.product_name,
                                   round(sum(oitl.order_item_subtotal),2) as Totalrevenue_permonth,  
                                   count(oitl.order_item_subtotal) as NumberOforders_permonth, 
                                   round(avg(oitl.order_item_subtotal),2) as avgsales_permonth,
                                   rank() over(partition by substring(otl.order_date,0,7) order by round(sum(oitl.order_item_subtotal),2) desc) ranking_order
                                   from orders otl,order_items oitl,products pdtl where otl.order_id=oitl.order_item_order_id
                                   and oitl.order_item_product_id = pdtl.product_id and CAST(otl.order_date AS DATE)>="2013-07-01" and CAST(otl.order_date AS DATE)<="2013-12-31"
                                   group by substring(otl.order_date,0,7),pdtl.product_name) s 
                                   WHERE ranking_order<=5 ORDER BY 1, 6''')
productsSalesDatafortop5revenue.coalesce(1).write.csv("/FileStore/tables/productsSalesDatafortop5revenue", compression="none", header ='true')
productsSalesDatafortop5revenue.show(30,False)

+----------+---------------------------------------------+---------------------+-----------------------+-----------------+-------------+
ordermonth|product_name |Totalrevenue_permonth|NumberOforders_permonth|avgsales_permonth|ranking_order|
+----------+---------------------------------------------+---------------------+-----------------------+-----------------+-------------+
2013-07 |Field & Stream Sportsman 16 Gun Fire Safe |155592.22 |389 |399.98 |1 |
2013-07 |Perfect Fitness Perfect Rip Deck |102582.9 |565 |181.56 |2 |
2013-07 |Diamondback Women's Serene Classic Comfort Bi|95993.6 |320 |299.98 |3 |
2013-07 |Nike Men's Free 5.0+ Running Shoe |78292.17 |271 |288.9 |4 |
2013-07 |O'Brien Men's Neoprene Life Vest |67722.9 |429 |157.86 |5 |
2013-08 |Field & Stream Sportsman 16 Gun Fire Safe |562371.88 |1406 |399.98 |1 |
2013-08 |Perfect Fitness Perfect Rip Deck |365279.11 |2019 |180.92 |2 |
2013-08 |Diamondback Women's Serene Classic Comfort Bi|332677.82 |1109 |299.98 |3 |
2013-08 |Nike Men's Free 5.0+ Running Shoe |292870.71 |999 |293.16 |4 |
2013-08 |Pelican Sunstream 100 Kayak |261986.9 |1310 |199.99 |5 |
2013-09 |Field & Stream Sportsman 16 Gun Fire Safe |583170.84 |1458 |399.98 |1 |
2013-09 |Perfect Fitness Perfect Rip Deck |369118.47 |2049 |180.15 |2 |
2013-09 |Diamondback Women's Serene Classic Comfort Bi|352476.5 |1175 |299.98 |3 |
2013-09 |Nike Men's Free 5.0+ Running Shoe |340165.98 |1123 |302.91 |4 |
2013-09 |Pelican Sunstream 100 Kayak |268586.57 |1343 |199.99 |5 |
2013-10 |Field & Stream Sportsman 16 Gun Fire Safe |524773.76 |1312 |399.98 |1 |
2013-10 |Perfect Fitness Perfect Rip Deck |341883.01 |1925 |177.6 |2 |
2013-10 |Diamondback Women's Serene Classic Comfort Bi|310779.28 |1036 |299.98 |3 |
2013-10 |Nike Men's Free 5.0+ Running Shoe |274672.53 |931 |295.03 |4 |
2013-10 |Nike Men's Dri-FIT Victory Golf Polo |247050.0 |1641 |150.55 |5 |
2013-11 |Field & Stream Sportsman 16 Gun Fire Safe |631568.42 |1579 |399.98 |1 |
2013-11 |Perfect Fitness Perfect Rip Deck |404632.55 |2236 |180.96 |2 |
2013-11 |Diamondback Women's Serene Classic Comfort Bi|389674.02 |1299 |299.98 |3 |
2013-11 |Nike Men's Free 5.0+ Running Shoe |343365.66 |1125 |305.21 |4 |
2013-11 |Nike Men's Dri-FIT Victory Golf Polo |291500.0 |1977 |147.45 |5 |
2013-12 |Field & Stream Sportsman 16 Gun Fire Safe |618769.06 |1547 |399.98 |1 |
2013-12 |Perfect Fitness Perfect Rip Deck |358920.17 |2003 |179.19 |2 |
2013-12 |Diamondback Women's Serene Classic Comfort Bi|350676.62 |1169 |299.98 |3 |
2013-12 |Nike Men's Free 5.0+ Running Shoe |312068.79 |1026 |304.16 |4 |
2013-12 |Pelican Sunstream 100 Kayak |262786.86 |1314 |199.99 |5 |
+----------+---------------------------------------------+---------------------+-----------------------+-----------------+-------------+

###### Problem statement-6:Get sales data for each product category

In [26]:
productsSalesDataPercategory = hiveContext.sql('''select CAST(pdtl.product_category_id AS INT) as categories,
                                   round(sum(oitl.order_item_subtotal),2) as orderrevenue,  
                                   count(oitl.order_item_subtotal) as NumberOforders, 
                                   round(avg(oitl.order_item_subtotal),2) as avgsales 
                                   from orders otl join order_items oitl on otl.order_id=oitl.order_item_order_id
                                   join products pdtl on oitl.order_item_product_id = pdtl.product_id
                                   group by categories
                                   order by categories ''')
productsSalesDataPercategory.coalesce(1).write.csv("/FileStore/tables/productsSalesDataPercategory", compression="none", header ='true')
productsSalesDataPercategory.show(60,False)

+----------+------------+--------------+--------+
categories|orderrevenue|NumberOforders|avgsales|
+----------+------------+--------------+--------+
2 |26477.05 |138 |191.86 |
3 |94057.15 |632 |148.82 |
4 |27099.33 |67 |404.47 |
5 |39464.79 |343 |115.06 |
6 |44585.09 |328 |135.93 |
7 |48360.73 |614 |78.76 |
9 |3694843.2 |12487 |295.9 |
10 |54895.53 |111 |494.55 |
11 |35601.44 |309 |115.22 |
12 |85205.41 |423 |201.43 |
13 |115355.25 |1127 |102.36 |
16 |20597.94 |68 |302.91 |
17 |4431942.66 |24551 |180.52 |
18 |2891757.54 |22246 |129.99 |
24 |3147800.0 |21035 |149.65 |
26 |151706.2 |1201 |126.32 |
29 |1309522.02 |10984 |119.22 |
30 |47035.8 |283 |166.2 |
31 |44545.97 |181 |246.11 |
32 |34969.75 |441 |79.3 |
33 |107998.0 |524 |206.1 |
34 |10369.39 |61 |169.99 |
35 |116695.39 |1070 |109.06 |
36 |77098.16 |1475 |52.27 |
37 |255679.39 |2029 |126.01 |
38 |98797.58 |384 |257.29 |
40 |133671.51 |1780 |75.1 |
41 |68721.78 |974 |70.56 |
43 |4118425.42 |13729 |299.98 |
44 |56848.42 |440 |129.2 |
45 |6929653.5 |17325 |399.98 |
46 |2888993.94 |19298 |149.7 |
48 |3113844.6 |15540 |200.38 |
+----------+------------+--------------+--------+

###### Problem statement-7:Get monthly sales data from Top5 revenue generating categories

In [28]:
productsSalesDatatop5categories = hiveContext.sql('''select * from (select substring(otl.order_date,0,7) as ordermonth,pdtl.product_category_id as categories,
                                   round(sum(oitl.order_item_subtotal),2) as Totalrevenue_permonth,  
                                   count(oitl.order_item_subtotal) as NumberOforders_permonth, 
                                   round(avg(oitl.order_item_subtotal),2) as avgsales_permonth,
                                   rank() over(partition by substring(otl.order_date,0,7) order by round(sum(oitl.order_item_subtotal),2) desc) ranking_order
                                   from orders otl,order_items oitl,products pdtl where otl.order_id=oitl.order_item_order_id
                                   and oitl.order_item_product_id = pdtl.product_id 
                                   group by substring(otl.order_date,0,7),pdtl.product_category_id) s 
                                   WHERE ranking_order<=5 ORDER BY 1, 6''')
productsSalesDatatop5categories.coalesce(1).write.csv("/FileStore/tables/productsSalesDatatop5categories", compression="none", header ='true')
productsSalesDatatop5categories.show(30,False)

+----------+----------+---------------------+-----------------------+-----------------+-------------+
ordermonth|categories|Totalrevenue_permonth|NumberOforders_permonth|avgsales_permonth|ranking_order|
+----------+----------+---------------------+-----------------------+-----------------+-------------+
2013-07 |45 |155592.22 |389 |399.98 |1 |
2013-07 |17 |103182.88 |567 |181.98 |2 |
2013-07 |43 |95993.6 |320 |299.98 |3 |
2013-07 |9 |79132.17 |282 |280.61 |4 |
2013-07 |46 |67722.9 |429 |157.86 |5 |
2013-08 |45 |562371.88 |1406 |399.98 |1 |
2013-08 |17 |367079.05 |2025 |181.27 |2 |
2013-08 |43 |332677.82 |1109 |299.98 |3 |
2013-08 |9 |295210.71 |1023 |288.57 |4 |
2013-08 |48 |262686.88 |1312 |200.22 |5 |
2013-09 |45 |583170.84 |1458 |399.98 |1 |
2013-09 |17 |370318.43 |2053 |180.38 |2 |
2013-09 |43 |352476.5 |1175 |299.98 |3 |
2013-09 |9 |342925.98 |1154 |297.16 |4 |
2013-09 |48 |269636.54 |1346 |200.32 |5 |
2013-10 |45 |524773.76 |1312 |399.98 |1 |
2013-10 |17 |343382.96 |1930 |177.92 |2 |
2013-10 |43 |310779.28 |1036 |299.98 |3 |
2013-10 |9 |276742.53 |958 |288.88 |4 |
2013-10 |24 |247050.0 |1641 |150.55 |5 |
2013-11 |45 |631568.42 |1579 |399.98 |1 |
2013-11 |17 |404632.55 |2236 |180.96 |2 |
2013-11 |43 |389674.02 |1299 |299.98 |3 |
2013-11 |9 |346395.66 |1159 |298.87 |4 |
2013-11 |24 |291500.0 |1977 |147.45 |5 |
2013-12 |45 |618769.06 |1547 |399.98 |1 |
2013-12 |17 |359820.14 |2006 |179.37 |2 |
2013-12 |43 |350676.62 |1169 |299.98 |3 |
2013-12 |9 |313868.79 |1047 |299.78 |4 |
2013-12 |48 |264186.82 |1318 |200.45 |5 |
+----------+----------+---------------------+-----------------------+-----------------+-------------+
only showing top 30 rows

###### Problem statement-8:Get highest product price for each category

In [30]:
productsSalesDatahighprice = hiveContext.sql('''select CAST(pdtl.product_category_id AS INT) as categories,pdtl.product_name,
                                   max(CAST(pdtl.product_price AS float)) as highestproductprice
                                   from products pdtl join
                                    (SELECT CAST(product_category_id AS INT) as categories, max(CAST(product_price AS float)) as highestproductprice
                                      FROM products 
                                      GROUP BY categories ORDER BY categories  ) cag 
                                      on CAST(pdtl.product_category_id AS INT)=cag.categories
                                    where pdtl.product_price = cag.highestproductprice
                                   group by CAST(pdtl.product_category_id AS INT),pdtl.product_name
                                   order by CAST(pdtl.product_category_id AS INT) ''')
productsSalesDatahighprice.coalesce(1).write.csv("/FileStore/tables/productsSalesDatahighprice", compression="none", header ='true')
productsSalesDatahighprice.show(60,False)

+----------+---------------------------------------------+-------------------+
categories|product_name |highestproductprice|
+----------+---------------------------------------------+-------------------+
2 |Riddell Youth 360 Custom Football Helmet |299.99 |
3 |Quik Shade Summit SX170 10 FT. x 10 FT. Canop|199.99 |
4 |SOLE F85 Treadmill |1799.99 |
5 |Goaliath 54" In-Ground Basketball Hoop with P|499.99 |
6 |YETI Tundra 65 Chest Cooler |399.99 |
7 |Stiga Master Series ST3100 Competition Indoor|329.99 |
8 |YETI Tundra 65 Chest Cooler |399.99 |
9 |Goaliath 54" In-Ground Basketball Hoop with P|499.99 |
10 |SOLE E35 Elliptical |1999.99 |
11 |Marcy Diamond 9010 Smith Cage |799.99 |
12 |Fitness Gear Pro Utility Bench |179.99 |
13 |Everlast PowerCore 2 Station Stand and Bag Ki|199.99 |
15 |Manduka PRO Yoga Mat |100.0 |
16 |Fitness Gear Pro Half Rack |349.99 |
17 |Total Gym 1900 |399.99 |
18 |PUMA Men's evoPOWER 1 Tricks FG Soccer Cleat |189.99 |
19 |PUMA Men's evoPOWER 1 Tricks FG Soccer Cleat |189.99 |
20 |Garmin Forerunner 220 GPS Watch |249.99 |
21 |Nike Kids' Grade School LeBron XI Basketball |139.99 |
22 |SOLE F85 Treadmill |1799.99 |
24 |Callaway X Hot Laser Rangefinder |229.99 |
25 |Callaway X Hot Laser Rangefinder |229.99 |
26 |adidas Men's Germany Home Soccer Jersey |90.0 |
26 |adidas Men's Germany Black/Red Away Match Soc|90.0 |
27 |adidas Men's Germany Black/Red Away Match Soc|90.0 |
27 |adidas Men's Germany Home Soccer Jersey |90.0 |
29 |Nike Men's Kobe IX Elite Low Basketball Shoe |199.99 |
30 |Merrell Men's Moab Rover Mid Waterproof Hikin|139.99 |
30 |Merrell Men's Moab Rover Mid Waterproof Wide |139.99 |
31 |TaylorMade SLDR Irons - (Steel) 4-PW, AW |899.99 |
32 |Callaway Women's Solaire Gems 20-Piece Comple|999.99 |
33 |Nike TW 14 Mesh Golf Shoes |169.99 |
34 |Ogio Race Golf Shoes |169.99 |
35 |Club Glove Last Bag |299.99 |
35 |Titleist Club Glove Travel Cover |299.99 |
36 |Hirzl Women's Trust Control Golf Glove |24.99 |
36 |Hirzl Women's Trust Feel Golf Glove |24.99 |
36 |Glove It Women's Violet Bling Golf Glove |24.99 |
37 |Titleist Pro V1 Double Number Golf Balls |51.99 |
37 |Titleist Pro V1x Personalized Golf Balls |51.99 |
37 |Titleist Pro V1x High Numbers Personalized Go|51.99 |
37 |Titleist Pro V1 Personalized Golf Balls |51.99 |
37 |Titleist Pro V1 High Numbers Personalized Gol|51.99 |
38 |Bushnell Pro X7 Jolt Slope Rangefinder |599.99 |
39 |Cobra Junior Kids' Complete Set (Height 53'' |199.99 |
40 |Team Golf Washington Redskins Putter Grip |24.99 |
40 |Team Golf New England Patriots Putter Grip |24.99 |
40 |Team Golf Detroit Tigers Putter Grip |24.99 |
40 |Team Golf Oakland Raiders Putter Grip |24.99 |
40 |Team Golf Missouri Tigers Putter Grip |24.99 |
40 |Team Golf Minnesota Vikings Putter Grip |24.99 |
40 |Team Golf St. Louis Cardinals Putter Grip |24.99 |
40 |Team Golf Dallas Cowboys Putter Grip |24.99 |
40 |Team Golf San Francisco 49ers Putter Grip |24.99 |
40 |Team Golf Pittsburgh Steelers Putter Grip |24.99 |
40 |Team Golf New York Giants Putter Grip |24.99 |
40 |Team Golf Penn State Nittany Lions Putter Gri|24.99 |
40 |Team Golf Wisconsin Badgers Putter Grip |24.99 |
40 |Team Golf Chicago Cubs Putter Grip |24.99 |
40 |Team Golf Chicago Bears Putter Grip |24.99 |
+----------+---------------------------------------------+-------------------+
only showing top 60 rows

###### Problem statement-9: Get the sales data of best-selling and the second best-selling products in every category

In [32]:
productSalesDatabstsellcategories = hiveContext.sql('''select * from (select CAST(pdtl.product_category_id AS INT) as categories,pdtl.product_name,
                                   round(sum(oitl.order_item_subtotal),2) as Totalrevenue,  
                                   count(oitl.order_item_subtotal) as NumberOforders, 
                                   round(avg(oitl.order_item_subtotal),2) as avgsales,
                                   rank() over(partition by CAST(pdtl.product_category_id AS INT) order by round(sum(oitl.order_item_subtotal),2) desc) bestselling_order
                                   from orders otl,order_items oitl,products pdtl where otl.order_id=oitl.order_item_order_id
                                   and oitl.order_item_product_id = pdtl.product_id 
                                   group by CAST(pdtl.product_category_id AS INT),pdtl.product_name) s 
                                   WHERE bestselling_order<=2 ORDER BY 1, 5 desc''')
productSalesDatabstsellcategories.coalesce(1).write.csv("/FileStore/tables/productSalesDatabstsellcategories", compression="none", header ='true')
productSalesDatabstsellcategories.show(30,False)

+----------+---------------------------------------------+------------+--------------+--------+-----------------+
categories|product_name |Totalrevenue|NumberOforders|avgsales|bestselling_order|
+----------+---------------------------------------------+------------+--------------+--------+-----------------+
2 |Elevation Training Mask 2.0 |18477.69 |74 |249.7 |1 |
2 |Nike Men's Fingertrap Max Training Shoe |7999.36 |64 |124.99 |2 |
3 |adidas Men's F10 Messi TRX FG Soccer Cleat |56330.61 |305 |184.69 |1 |
3 |adidas Kids' F5 Messi FG Soccer Cleat |27327.19 |262 |104.3 |2 |
4 |SOLE E25 Elliptical |9999.9 |10 |999.99 |1 |
4 |Diamondback Boys' Insight 24 Performance Hybr|8699.71 |29 |299.99 |2 |
5 |Nike Kids' Grade School KD VI Basketball Shoe|18598.14 |63 |295.21 |2 |
5 |Under Armour Men's Tech II T-Shirt |20866.65 |280 |74.52 |1 |
6 |Nike Men's Comfort 2 Slide |44585.09 |328 |135.93 |1 |
7 |Nike Women's Legend V-Neck T-Shirt |20025.0 |279 |71.77 |1 |
7 |Nike Dri-FIT Crew Sock 6 Pack |19426.0 |308 |63.07 |2 |
9 |Nike Men's Free 5.0+ Running Shoe |3667633.2 |12169 |301.39 |1 |
9 |Nike Women's Tempo Shorts |27210.0 |318 |85.57 |2 |
10 |SOLE E35 Elliptical |29999.85 |15 |1999.99 |1 |
10 |GoPro HERO3+ Black Edition Camera |12799.68 |32 |399.99 |2 |
11 |Bowflex SelectTech 1090 Dumbbells |5999.9 |10 |599.99 |2 |
11 |Under Armour Hustle Storm Medium Duffle Bag |29601.54 |299 |99.0 |1 |
12 |Nike Women's Free 5.0 TR FIT PRT 4 Training S|19567.94 |70 |279.54 |2 |
12 |Under Armour Women's Micro G Skulpt Running S|46559.59 |287 |162.23 |1 |
13 |Under Armour Men's Compression EV SL Slide |36576.87 |280 |130.63 |1 |
13 |Under Armour Women's Ignite Slide |29398.81 |311 |94.53 |2 |
16 |Nike Men's Free TR 5.0 TB Training Shoe |20597.94 |68 |302.91 |1 |
17 |Total Gym 1400 |10799.64 |36 |299.99 |2 |
17 |Perfect Fitness Perfect Rip Deck |4421143.02 |24515 |180.34 |1 |
18 |Nike Men's CJ Elite 2 TD Football Cleat |2891757.54 |22246 |129.99 |1 |
24 |Nike Men's Dri-FIT Victory Golf Polo |3147800.0 |21035 |149.65 |1 |
26 |adidas Youth Germany Black/Red Away Match Soc|67830.0 |314 |216.02 |1 |
26 |TYR Boys' Team Digi Jammer |35191.2 |300 |117.3 |2 |
29 |Under Armour Girls' Toddler Spine Surge Runni|1269082.65 |10617 |119.53 |1 |
29 |Columbia Men's PFG Anchor Tough T-Shirt |27840.0 |304 |91.58 |2 |
+----------+---------------------------------------------+------------+--------------+--------+-----------------+
only showing top 30 rows

###### Problem statement-10:Get TOP5 highest selling product for each year

In [34]:
productsSalesDatatop5peryear = hiveContext.sql('''select * from (select substring(otl.order_date,0,4) as orderyear,pdtl.product_name,
                                   round(sum(oitl.order_item_subtotal),2) as Totalrevenue,  
                                   count(oitl.order_item_subtotal) as NumberOforders, 
                                   round(avg(oitl.order_item_subtotal),2) as avgsales,
                                   rank() over(partition by substring(otl.order_date,0,4) order by round(sum(oitl.order_item_subtotal),2) desc) ranking_order
                                   from orders otl,order_items oitl,products pdtl where otl.order_id=oitl.order_item_order_id
                                   and oitl.order_item_product_id = pdtl.product_id 
                                   group by substring(otl.order_date,0,4),pdtl.product_name) t 
                                   WHERE ranking_order<=5 ORDER BY 1, 6 ''')
productsSalesDatatop5peryear.coalesce(1).write.csv("/FileStore/tables/productsSalesDatatop5peryear", compression="none", header ='true')
productsSalesDatatop5peryear.show(30,False)

+---------+---------------------------------------------+------------+--------------+--------+-------------+
orderyear|product_name |Totalrevenue|NumberOforders|avgsales|ranking_order|
+---------+---------------------------------------------+------------+--------------+--------+-------------+
2013 |Field & Stream Sportsman 16 Gun Fire Safe |3076246.18 |7691 |399.98 |1 |
2013 |Perfect Fitness Perfect Rip Deck |1942416.21 |10797 |179.9 |2 |
2013 |Diamondback Women's Serene Classic Comfort Bi|1832277.84 |6108 |299.98 |3 |
2013 |Nike Men's Free 5.0+ Running Shoe |1641435.84 |5475 |299.81 |4 |
2013 |Nike Men's Dri-FIT Victory Golf Polo |1393000.0 |9313 |149.58 |5 |
2014 |Field & Stream Sportsman 16 Gun Fire Safe |3853407.32 |9634 |399.98 |1 |
2014 |Perfect Fitness Perfect Rip Deck |2478726.81 |13718 |180.69 |2 |
2014 |Diamondback Women's Serene Classic Comfort Bi|2286147.58 |7621 |299.98 |3 |
2014 |Nike Men's Free 5.0+ Running Shoe |2026197.36 |6694 |302.69 |4 |
2014 |Nike Men's Dri-FIT Victory Golf Polo |1754800.0 |11722 |149.7 |5 |
+---------+---------------------------------------------+------------+--------------+--------+-------------+